<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Catch Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

### Задание
1. Заполните код в этой тетрадке 
2. Если вы проходите специализацию Яндеса и МФТИ, пошлите тетрадку в соответствующем Peer Review. <br> Если вы проходите курс ODS, выберите ответы в [веб-форме](https://docs.google.com/forms/d/1pLsegkAICL9PzOLyAeH9DmDOBfktte0l8JW75uWcTng). 

In [56]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from collections import Counter

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [57]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = "C:\\Users\\arxei\\Anaconda\\CourseraDeepLearning\\Course6\\UsersIdentification\\kaggle"

In [58]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [59]:
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (253561, 21)
Test shape: (82797, 20)


In [60]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [61]:
train_test_df = pd.concat([train_df, test_df])

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [62]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   site1   253561 non-null  int64  
 1   time1   253561 non-null  object 
 2   site2   250098 non-null  float64
 3   time2   250098 non-null  object 
 4   site3   246919 non-null  float64
 5   time3   246919 non-null  object 
 6   site4   244321 non-null  float64
 7   time4   244321 non-null  object 
 8   site5   241829 non-null  float64
 9   time5   241829 non-null  object 
 10  site6   239495 non-null  float64
 11  time6   239495 non-null  object 
 12  site7   237297 non-null  float64
 13  time7   237297 non-null  object 
 14  site8   235224 non-null  float64
 15  time8   235224 non-null  object 
 16  site9   233084 non-null  float64
 17  time9   233084 non-null  object 
 18  site10  231052 non-null  float64
 19  time10  231052 non-null  object 
 20  target  253561 non-null  int64  
dtypes: float64

In [63]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [64]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   site1   82797 non-null  int64  
 1   time1   82797 non-null  object 
 2   site2   81308 non-null  float64
 3   time2   81308 non-null  object 
 4   site3   80075 non-null  float64
 5   time3   80075 non-null  object 
 6   site4   79182 non-null  float64
 7   time4   79182 non-null  object 
 8   site5   78341 non-null  float64
 9   time5   78341 non-null  object 
 10  site6   77566 non-null  float64
 11  time6   77566 non-null  object 
 12  site7   76840 non-null  float64
 13  time7   76840 non-null  object 
 14  site8   76151 non-null  float64
 15  time8   76151 non-null  object 
 16  site9   75484 non-null  float64
 17  time9   75484 non-null  object 
 18  site10  74806 non-null  float64
 19  time10  74806 non-null  object 
dtypes: float64(9), int64(1), object(10)
memory usage: 13.3+ MB


**В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.**

In [65]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [66]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [67]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178
6,570,21,570,21,21,0,0,0,0,0
7,803,23,5956,17513,37,21,803,17514,17514,17514
8,22,21,29,5041,14422,23,21,5041,14421,14421
9,668,940,942,941,941,942,940,23,21,22


In [68]:
print("Train-test shape:", train_test_df_sites.shape)

Train-test shape: (336358, 10)


**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу *train_test_df_sites*, потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [69]:
def make_csr_matrix(data, session_length=10): 
    # На пересечении строки  𝑖  и столбца  𝑗  будет стоять число  𝑛_𝑖𝑗
    # – cколько раз сайт  𝑗  встретился в сессии номер  𝑖
    indptr = [0]
    indices = []
    prematrix = []
    vocabulary = {}
    for one_session in data:
        for site_id in one_session:
            index = vocabulary.setdefault(site_id, site_id)
            indices.append(index)
            prematrix.append(1)
        indptr.append(len(indices))
    return csr_matrix((prematrix, indices, indptr), dtype=int)

In [70]:
train_test_sparse = make_csr_matrix(np.array(train_test_df_sites))[:,1::]
print(train_test_sparse)

  (0, 717)	1
  (1, 889)	1
  (1, 940)	1
  (1, 3846)	1
  (1, 940)	1
  (1, 941)	1
  (1, 3845)	1
  (1, 3846)	1
  (1, 3845)	1
  (1, 1515)	1
  (1, 1517)	1
  (2, 14768)	1
  (2, 38)	1
  (2, 14767)	1
  (2, 14768)	1
  (2, 36)	1
  (2, 38)	1
  (2, 14767)	1
  (2, 14767)	1
  (2, 14767)	1
  (2, 14767)	1
  (3, 781)	1
  (3, 781)	1
  (3, 781)	1
  (3, 781)	1
  :	:
  (336355, 6778)	1
  (336355, 29)	1
  (336355, 20)	1
  (336355, 22)	1
  (336355, 6779)	1
  (336356, 5827)	1
  (336356, 22)	1
  (336356, 20)	1
  (336356, 803)	1
  (336356, 20)	1
  (336356, 3349)	1
  (336356, 22)	1
  (336356, 893)	1
  (336356, 20)	1
  (336356, 960)	1
  (336357, 20)	1
  (336357, 1097)	1
  (336357, 1097)	1
  (336357, 1097)	1
  (336357, 1097)	1
  (336357, 1097)	1
  (336357, 1097)	1
  (336357, 1097)	1
  (336357, 1097)	1
  (336357, 1097)	1


In [71]:
X_train_sparse = train_test_sparse[:train_df.shape[0]:,:]
X_test_sparse = train_test_sparse[train_df.shape[0]::,:]
y = train_df["target"]
print(X_train_sparse.shape[0] == train_df.shape[0])
print(X_test_sparse.shape[0] == test_df.shape[0])
print(X_train_sparse.shape)

True
True
(253561, 48371)


**<font color='red'>Вопрос 1. </font> Выведите размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*.**

In [72]:
print(X_train_sparse.shape[0], X_train_sparse.shape[1], X_test_sparse.shape[0], X_test_sparse.shape[1])

253561 48371 82797 48371


**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [73]:
PATH_TO_DATA = "C:\\Users\\arxei\\Anaconda\\CourseraDeepLearning\\Course6\\UsersIdentification"

In [74]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, "kaggle", 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Разобьем обучающую выборку на 2 части в пропорции 7/3, причем не перемешивая. Исходные данные упорядочены по времени, тестовая выборка по времени четко отделена от обучающей, это же соблюдем и здесь.**

In [75]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

In [76]:
print(X_train.shape)
print(X_valid.shape)

(177492, 48371)
(76069, 48371)


**Создайте объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставьте по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучите  модель на выборке `(X_train, y_train)`.**

In [77]:
%%time
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train, y_train)

Wall time: 456 ms


SGDClassifier(loss='log', n_jobs=-1, random_state=17)

**Сделайте прогноз в виде предсказанных вероятностей того, что это сессия Элис, на отложенной выборке *(X_valid, y_valid)*.**

In [78]:
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)
print(logit_valid_pred_proba)
print(len(logit_valid_pred_proba[logit_valid_pred_proba > 0.5]))

[[9.11911413e-01 8.80885865e-02]
 [9.99703507e-01 2.96493343e-04]
 [9.99872654e-01 1.27345522e-04]
 ...
 [9.87751880e-01 1.22481202e-02]
 [9.99943934e-01 5.60663172e-05]
 [9.88685254e-01 1.13147460e-02]]
76069


**<font color='red'>Вопрос 2. </font> Посчитайте ROC AUC логистической регрессии, обученной с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя.**

In [79]:
print(round(roc_auc_score(y_valid, logit_valid_pred_proba[:, 1]), 3))

0.934


**Сделайте прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [24]:
%%time
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_sparse, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)

Wall time: 545 ms


**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

In [54]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [69]:
write_to_submission_file(logit_test_pred_proba[:, 1], os.path.join(PATH_TO_DATA, "kaggle", "[YDF & MIPT] Coursera_Elizarov"))

## Мои предсказания

### GridSearcv (worse)

In [102]:
sgd_params = {"alpha": np.linspace(0.000005, 0.00001, 10),
              "max_iter": np.linspace(600, 1200, 4, dtype=int)}
print(sgd_params)

{'alpha': array([5.00000000e-06, 5.55555556e-06, 6.11111111e-06, 6.66666667e-06,
       7.22222222e-06, 7.77777778e-06, 8.33333333e-06, 8.88888889e-06,
       9.44444444e-06, 1.00000000e-05]), 'max_iter': array([ 600,  800, 1000, 1200])}


In [103]:
%%time
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_grid_searcher = GridSearchCV(sgd_logit, param_grid=sgd_params, scoring='roc_auc')
sgd_grid_searcher.fit(X_train_sparse, y)

Wall time: 2min 31s


GridSearchCV(estimator=SGDClassifier(loss='log', n_jobs=-1, random_state=17),
             param_grid={'alpha': array([5.00000000e-06, 5.55555556e-06, 6.11111111e-06, 6.66666667e-06,
       7.22222222e-06, 7.77777778e-06, 8.33333333e-06, 8.88888889e-06,
       9.44444444e-06, 1.00000000e-05]),
                         'max_iter': array([ 600,  800, 1000, 1200])},
             scoring='roc_auc')

In [104]:
print("Best score:", round(sgd_grid_searcher.best_score_, 3)) #0.953

alpha_best = sgd_grid_searcher.best_params_['alpha']
max_iter_best = sgd_grid_searcher.best_params_['max_iter']
print("Best alpha:", alpha_best)
print("Best max iter:", max_iter_best)

Best score: 0.959
Best alpha: 1e-05
Best max iter: 600


In [105]:
sgd_logit_best = SGDClassifier(loss='log', alpha=alpha_best, max_iter=max_iter_best, random_state=17, n_jobs=-1)
sgd_logit_best.fit(X_train_sparse, y)
logit_best_test_pred_proba = sgd_logit_best.predict_proba(X_test_sparse)

In [106]:
write_to_submission_file(logit_best_test_pred_proba[:, 1], os.path.join(PATH_TO_DATA, "[YDF & MIPT] Coursera_Elizarov"))

### My features (failed)

In [25]:
PATH_TO_DATA = "C:\\Users\\arxei\\Anaconda\\CourseraDeepLearning\\Course6\\UsersIdentification"

In [27]:
def get_top_sites(site_freq_Nusers, top=20): # returns names of (top=20) most visited sites
    site_freq_Nusers_sorted = dict(sorted(site_freq_Nusers.items(), key=lambda item: item[1][1], reverse=True))
    return list(site_freq_Nusers_sorted.keys())[:top:]


def get_top_domens(site_freq_Nusers, threshold=0.15):   
        # returns domens which were visited 10% (or more) often as most visited domen
        
    site_names = list(site_freq_Nusers.keys())
    domen_names = list([site_names[i][site_names[i].rfind('.')+1::] for i in range(len(site_names))])
    counter_domen_names = Counter(domen_names)
    max_freq = list(counter_domen_names.values())[0]
    top_domen_names = [site for site, freq in counter_domen_names.items() if (freq / max_freq) > threshold]
    return top_domen_names


def count_unique_sites_among_session(one_ses_dict):
    keys = ["site" + str(i) for i in np.arange(1, 11)]
    return len(np.unique(np.array([one_ses_dict[key] for key in keys if one_ses_dict[key] > 0])))

In [28]:
def feature_engineering(path_to_alice, site_freq_path, session_length=10):
    csv_file = os.path.join(path_to_alice)
    
    with open(site_freq_path, 'rb') as f:
        site_to_id_freq = pickle.load(f)
    
    most_popular_sites = get_top_sites(site_to_id_freq)
    top_domen_names = get_top_domens(site_to_id_freq)
    
    df_columns = ["site" + str(i) for i in np.arange(1, session_length + 1)]
    df_columns.extend(["session_timespan", "#unique_sites", "start_hour", "day_of_week"])
    df_columns.extend(top_domen_names)
    df_columns.extend(["#popular_sites", "month_start"])
    
    dataframe = pd.DataFrame(columns=df_columns)
    
    all_sessions_data = []
    one_session_dict = dict.fromkeys(df_columns, 0)
    
    
    min_sec = np.Inf
    max_sec = -1            
    min_hour = np.Inf
    min_day = np.Inf
    
    session_counter = 0
        
    data = pd.read_csv(csv_file, header=0, names=["Date_time", "Sites"])
    date_time = pd.to_datetime(data["Date_time"])
    sites = data["Sites"]
        
    for i in range(len(sites)):
        
        cur_site = sites[i]
        cur_time = date_time[i]
        
        if cur_site == 0:
            pass
        else:
            
            month = cur_time.month
            one_session_dict["month_start"] = month
            
            site_id = site_to_id_freq.get(cur_site)[0]
            one_session_dict[f"site{session_counter + 1}"] = site_id

            site_domen = cur_site[cur_site.rfind('.')+1::]
            if site_domen in top_domen_names:
                one_session_dict[site_domen] += 1

            if cur_site in most_popular_sites:
                one_session_dict["#popular_sites"] += 1
            
            cur_seconds = (cur_time - pd.datetime(2010,1,1)).total_seconds()
            if cur_seconds < min_sec:
                min_sec = cur_seconds
            if cur_seconds > max_sec:
                max_sec = cur_seconds
                
                
            if cur_time.hour < min_hour:
                min_hour = cur_time.hour
            if cur_time.weekday() < min_day:
                min_day = cur_time.weekday()
        
                
        session_counter += 1
            
        if session_counter % session_length == 0:
            
            unique_sites = count_unique_sites_among_session(one_session_dict)
            
            one_session_dict["session_timespan"] = int(max_sec - min_sec)
            one_session_dict["#unique_sites"] = unique_sites
            one_session_dict["start_hour"] = min_hour
            one_session_dict["day_of_week"] = min_day
            
            all_sessions_data.append(one_session_dict.values())
            one_session_dict = dict.fromkeys(one_session_dict, 0)
                
            session_counter = 0
                
                
    if session_counter % session_length != 0:
        unique_sites = count_unique_sites_among_session(one_session_dict)
            
        one_session_dict["session_timespan"] = int(max_sec - min_sec)
        one_session_dict["#unique_sites"] = unique_sites
        one_session_dict["start_hour"] = min_hour
        one_session_dict["day_of_week"] = min_day
        
        all_sessions_data.append(list(one_session_dict.values()))
        one_session_dict = dict.fromkeys(one_session_dict, 0)
            
        session_counter = 0
      
    dataframe = pd.DataFrame(all_sessions_data, columns=df_columns)
    # print(dataframe)
    
    return dataframe   

In [29]:
alice_log = feature_engineering(os.path.join(PATH_TO_DATA, "kaggle", "Alice_log.csv"), 
                    os.path.join(PATH_TO_DATA, "site_freq_150users.pkl"), 
                                 session_length=10)

In [30]:
sum(train_df['target'])

2297

In [189]:
print(selected_features_150users.shape)
print(np.array(alice_log).shape)

(137019, 21)
(2277, 20)


In [ ]:
my_train_test_df = 

In [182]:
my_train_sparse = make_csr_matrix(np.array(selected_features_150users))[:,1::]
my_test_sparse = make_csr_matrix(np.array(alice_log))[:,1::]

In [181]:
print(my_train_sparse.shape)
print(my_test_sparse.shape)

(137019, 9404616)
(2277, 36898960)


In [61]:
test = pd.read_csv(os.path.join(PATH_TO_DATA, "Alice_log.csv"))

### Some features from kaggle (worse)

In [109]:
def time_operations(df):
    if 'target' in df.columns:
        df['number_of_sites'] = ((df.notna().sum(axis=1)) / 2) - 1
    else:
        df['number_of_sites'] = ((df.notna().sum(axis=1)) / 2) - 0.5
    for col in [f"time{i}" for i in range(1,11)]:
        df[col] = pd.to_datetime(df[col])
    df['session_strt'] = df[[f"time{i}" for i in range(1,11)]].min(axis=1)
    df['session_end'] = df[[f"time{i}" for i in range(1,11)]].max(axis=1)
    df['session_len'] = (df['session_end'] - df['session_strt']).dt.seconds
    df['weekday'] = df['session_strt'].dt.dayofweek
    df['weekday'] = df['weekday'].astype('category')
    df['start_hour'] = df['session_strt'].dt.hour
    df['start_hour'] = df['start_hour'].astype('category')
    df['end_hour'] = df['session_end'].dt.hour
    df['end_hour'] = df['end_hour'].astype('category')
    df['day'] = df['session_strt'].dt.day
    df['day'] = df['day'].astype('category')
    df['minute'] = df['session_strt'].dt.minute
    df['month'] = df['session_strt'].dt.month
    df = df.sort_values('session_strt', ignore_index=True)
    df = df.reset_index()
    df = df.rename(columns={'index':'time_index'})
    # df = df.sort_values('session_id', ignore_index=True)
    return df

In [110]:
train_df_kaggle = time_operations(train_df)
test_df_kaggle = time_operations(test_df)
# train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [111]:
train_test_df_kaggle = pd.concat([train_df_kaggle, test_df_kaggle])
train_test_df_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336358 entries, 0 to 82796
Data columns (total 32 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   time_index       336358 non-null  int64         
 1   site1            336358 non-null  int64         
 2   time1            336358 non-null  datetime64[ns]
 3   site2            331406 non-null  float64       
 4   time2            331406 non-null  datetime64[ns]
 5   site3            326994 non-null  float64       
 6   time3            326994 non-null  datetime64[ns]
 7   site4            323503 non-null  float64       
 8   time4            323503 non-null  datetime64[ns]
 9   site5            320170 non-null  float64       
 10  time5            320170 non-null  datetime64[ns]
 11  site6            317061 non-null  float64       
 12  time6            317061 non-null  datetime64[ns]
 13  site7            314137 non-null  float64       
 14  time7            3141

In [112]:
train_test_df_kaggle.head()

,time_index,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,number_of_sites,session_strt,session_end,session_len,weekday,start_hour,end_hour,day,minute,month
0,0,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,...,6.5,2013-01-12 08:05:57,2013-01-12 08:05:57,0,5,8,8,12,5,1
1,1,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,...,8.5,2013-01-12 08:37:23,2013-01-12 09:07:09,1786,5,8,9,12,37,1
2,2,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,...,14.5,2013-01-12 08:50:13,2013-01-12 08:50:17,4,5,8,8,12,50,1
3,3,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,...,14.5,2013-01-12 08:50:17,2013-01-12 08:50:20,3,5,8,8,12,50,1
4,4,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,...,14.5,2013-01-12 08:50:20,2013-01-12 08:50:22,2,5,8,8,12,50,1


In [113]:
df_columns = ['site%d' % i for i in range(1, 11)]
df_columns.extend(["number_of_sites", "session_len", "weekday", "minute"])
train_test_df_kaggle = train_test_df_kaggle[df_columns].fillna(0)
train_test_df_kaggle.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,number_of_sites,session_len,weekday,minute
0,56,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,0,5,5
1,56,55.0,56.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,8.5,1786,5,37
2,946,946.0,951.0,946.0,946.0,945.0,948.0,784.0,949.0,946.0,14.5,4,5,50
3,945,948.0,949.0,948.0,945.0,946.0,947.0,945.0,946.0,946.0,14.5,3,5,50
4,947,950.0,948.0,947.0,950.0,952.0,946.0,951.0,946.0,947.0,14.5,2,5,50


In [114]:
train_test_df_kaggle = pd.concat([train_test_df_kaggle.drop(columns=["weekday"]), pd.get_dummies(train_test_df_kaggle["weekday"])], axis=1)
train_test_df_kaggle.columns

Index([          'site1',           'site2',           'site3',
                 'site4',           'site5',           'site6',
                 'site7',           'site8',           'site9',
                'site10', 'number_of_sites',     'session_len',
                'minute',                 0,                 1,
                       2,                 3,                 4,
                       5,                 6],
      dtype='object')

In [115]:
train_test_sparse = make_csr_matrix(np.array(train_test_df_kaggle))[:,1::]

In [116]:
X_train_sparse = train_test_sparse[:train_df.shape[0]:,:]
X_test_sparse = train_test_sparse[train_df.shape[0]::,:]
y = train_df["target"]
print(X_train_sparse.shape[0] == train_df.shape[0])
print(X_test_sparse.shape[0] == test_df.shape[0])
print(X_train_sparse.shape)

True
True
(253561, 48371)


In [117]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

In [118]:
%%time
sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train_sparse, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)

Wall time: 631 ms


In [119]:
write_to_submission_file(logit_test_pred_proba[:, 1], os.path.join(PATH_TO_DATA, "[YDF & MIPT] Coursera_Elizarov")) # 0.9

## Критерии оценки работы (только для Peer Review в специализации):
- Правильные ли получились размерности матриц в п. 1? (max. 2 балла)
- Правильным ли получилось значения ROC AUC в п. 2? (max. 4 балла)
- Побит ли бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)
- Побит ли бенчмарк "Logit +3 features" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)

**Основной код в другом отдельном ноутбуке, прикрепил сюда скриншот с kaggle**

<img src="Screenshot_1.png">

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в описание финального проекта (`html`, `pdf` или `ipynb`). Это только в случае, если вы проходите специализацию.
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям, отделив одного из пользователей от остальных – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов, также хороша [статья](https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending) Александра Дьяконова
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных веб-страницах Элис и остальными 1557 пользователями (*train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем большой тьюториал по Vowpal Wabbit и попробуем его в деле, на данных соревнования.